In [3]:
pip install folium

In [5]:
import requests
import folium
from geopy.distance import geodesic

def get_real_time_traffic_data(origin, destination, api_key):
    """
    Fetches real-time traffic data between the origin and destination using a traffic API (Google Maps API, for example).

    Parameters:
    - origin: tuple of (latitude, longitude) for the current user location
    - destination: tuple of (latitude, longitude) for the charging station
    - api_key: your API key for the traffic data service

    Returns:
    - The travel time with traffic considered
    """
    url = f"https://maps.googleapis.com/maps/api/directions/json"
    params = {
        'origin': f"{origin[0]},{origin[1]}",
        'destination': f"{destination[0]},{destination[1]}",
        'departure_time': 'now',  # This is used to get live traffic data
        'key': api_key
    }

    response = requests.get(url, params=params)
    data = response.json()

    if data['status'] == 'OK':
        # Extract travel time with traffic (in seconds)
        travel_time = data['routes'][0]['legs'][0]['duration_in_traffic']['value']
        return travel_time / 60  # Convert to minutes
    else:
        return None

def find_nearest_ev_station(user_location, charging_stations, api_key):
    """
    Finds the nearest EV charging station based on real-time traffic data.

    Parameters:
    - user_location: tuple of (latitude, longitude) representing the user's current location
    - charging_stations: list of tuples representing the (latitude, longitude) of available charging stations
    - api_key: API key for real-time traffic data

    Returns:
    - The nearest charging station considering real-time traffic
    """
    min_travel_time = float('inf')
    nearest_station = None

    for station in charging_stations:
        # Get the travel time to the station with traffic considered
        travel_time = get_real_time_traffic_data(user_location, station, api_key)

        if travel_time is not None and travel_time < min_travel_time:
            min_travel_time = travel_time
            nearest_station = station

    return nearest_station, min_travel_time

def create_map(user_location, charging_stations, nearest_station):
    """
    Creates and displays a map using folium showing the user location, EV stations, and the nearest station.

    Parameters:
    - user_location: tuple of (latitude, longitude) for the user's current location
    - charging_stations: list of tuples of available charging station locations
    - nearest_station: tuple representing the nearest charging station based on traffic
    """
    # Create a folium map centered at the user's location
    map_center = user_location
    ev_map = folium.Map(location=map_center, zoom_start=13)

    # Add marker for the user's location
    folium.Marker(location=user_location, popup="User Location", icon=folium.Icon(color='blue')).add_to(ev_map)

    # Add markers for each EV charging station
    for station in charging_stations:
        folium.Marker(location=station, popup="Charging Station", icon=folium.Icon(color='green')).add_to(ev_map)

    # Highlight the nearest station
    if nearest_station:
        folium.Marker(location=nearest_station, popup="Nearest Station", icon=folium.Icon(color='red')).add_to(ev_map)

    # Save map to an HTML file and display it
    ev_map.save("nearest_ev_station_map.html")
    return ev_map

# Example usage:
user_location = (37.7749, -122.4194)  # Example user location (San Francisco)
charging_stations = [(37.7849, -122.4294), (37.7649, -122.4094), (37.7549, -122.4394)]  # Example EV station locations

api_key = 'Insert your actual API key here'  # Insert your actual API key here
nearest_station, time = find_nearest_ev_station(user_location, charging_stations, api_key)

print(f"Nearest EV Station: {nearest_station}, Travel time: {time} minutes")

# Create and display the map with markers
ev_map = create_map(user_location, charging_stations, nearest_station)
ev_map


Nearest EV Station: (37.7649, -122.4094), Travel time: 4.733333333333333 minutes
